In [1]:
import pandas as pd
import numpy as np
import requests
import pdfplumber
import re
import statistics
from urllib.request import Request, urlopen
from io import StringIO, BytesIO

## Pdf urls and pages

In [7]:
abanca={1:{'link':'https://www.abanca.pt', 'page':None}, 
        2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf', 'page':[6,7]},
        3:{'link':'https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf', 'page':[6]}}

bankinter={1:{'link':'https://www.bankinter.pt', 'page':None},
           2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf', 'page':[8,9,10,11,12,13,14,15,16,17]},
           3:{'link': 'https://storage.googleapis.com/bank_price_pdfs/2_all_products_210412130350.pdf', 'page':[], 'notes':'image file'}}

bic={1:{'link':'https://www.bancobic.ao' ,'page':None},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE_0.pdf', 'page':[7,8,9,10,12,13]},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/3_all_products_210412130840.pdf', 'page':[], 'notes':'wrong file'}}

bai={1:{'link':'https://www.bancobai.ao', 'page':None},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0008_/0008_PRE.pdf', 'page':[], 'notes':'No housing credit products'},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/4_all_products_210412130952.pdf', 'page':[], 'notes':'No housing credit products'}}

ctt={1:{'link':'https://www.bancoctt.pt/', 'page':None},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf', 'page':[6]},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/5_all_products_210412131014.pdf', 'page':[6]}}

## Housing Credit Dictionary

In [3]:
house_credit_com = {'admin':['Comissões associadas a atos administrativos 4.1 Não realização da escritura',
                             'Alteração do local da escritura',
                             'Declarações de dívida',
                             'Mudança de regime de crédito',
                             'Declarações de dívida',
                             'Pedido de 2ª via de Caderneta Predial',
                             'Emissão de declarações não obrigatórias por lei',
                             'Emissão de 2ª vias de Declaração para efeitos de IRS – Urgente',
                             'Emissão de 2º vias de Declaração para efeitos de IRS',
                             'Emissão de 2ª vias de faturas',
                             'Declaração de Dívida para Fins Diversos',
                             'Declaração de Encargos com Prestações'],
                    'certificates':['Emolumentos do registo predial', 'registo predial',
                                    'Certidão permanente on-line'],
                    'debt_recovery':['Comissão de recuperação de valores em dívida', 'Prestação até 50.000 €',
                                    'Prestação > 50.000 €', 'Comissão de recuperação de valores em dívida',
                                    'Prestação > 50.000,00€', 'Prestação ≤ 50.000,00€'],
                    'displacement':['Comissão de deslocação', 'Até 100 Kms', '101 a 250 Kms', '> 250 Kms '],
                    'early_payment':['Comissão de reembolso antecipado parcial', 'Taxa fixa', 'Taxa variável', 
                                    'Taxa fixa', 'Comissão de reembolso antecipado total', 'Comissão de antecipação',
                                    '(pré.aviso 7 dias)', 'Comissão de compra antecipada', '(pré-aviso 10 dias)',
                                    'Comissão de Reembolso Antecipado Parcial',
                                    'Comissão de reembolso antecipado total'],
                    'evaluation':['Avaliação', 'Imóvel residencial', 
                                 'Garagens e arrecadações não anexas ao imóvel residencial', 'Avaliação do Imóvel'],
                    'formalization':['Comissão de formalização', 'Formalização'],
                    'process':['Processo', 'Abertura de Processo',
                              'Desistência ou não conclusão do processo por motivos imputáveis ao cliente'],
                    'inspections':['Vistorias', 'em caso de construção ou realização de obras'],
                    'reanalysis':['Reanálise'],
                    'settlement':['Comissão de Liquidação de Prestação', 'Liquidação de Prestação'],
                    'solicitors_notary':['Emolumentos notariais', 'Solicitadoria', 'Notiário'],
                    'statements':['Emissão de extratos de conta de empréstimos liquidados', 'extrato', 'extratos',
                                  'extrato de conta', 'extrato mensal'],
                    'taxes':['Imposto do Selo sobre concessão de crédito', 'imposto', 'imposto de selo', 'impostos'],
                    'termination':['Cessação da posição contratual', 'cessação', 'rescisão', 'encerramento']}

## Housing Credit Prices Scraping Function

In [4]:
def get_df(pdf,page, dictionary):
    doc = pdfplumber.open(pdf)
    prod_df=pd.DataFrame(doc.pages[page].extract_table())
    
    prices_col=find_prices(prod_df)
    index_col=find_index(prod_df, house_credit_com)
    
    prices_df=pd.DataFrame()
    prices_df['Commissions']=prod_df[index_col]
    prices_df['Prices']=prod_df[prices_col]
    
    return clean_df(prices_df)
    
def find_prices(df):
    col=''
    for x in range(df.shape[0]):
        for y in range(df.shape[1]):
            if 'Euros' in str(df[y][x]):
                col = y
    if col=='':
        col=3
    return col

def find_index(df,dictionary):
    index_col=list()
    for x in range(df.shape[0]):
        for y in range(find_prices(df)):
            if str(df[y][x])=='None' or str(df[y][x])=='' or 'Nota' in str(df[y][x]):
                pass
            else:
                words = [word for word in str(df[y][x]).split(' ')]
                for word in words:
                    for key in dictionary.keys():
                        for item in dictionary[key]:
                            if word in item:
                                index_col.append(y)
    return int(statistics.mode(index_col))

def clean_df(df):
    for x in range(df.shape[0]):
        for y in df.columns:
            if str(df[y][x])=='' or str(df[y][x])=='None':
                df[y][x]=np.nan
    return df.dropna(axis='columns', how='all').dropna(axis='rows', how='all')


def get_pdf(link):
        remote = urlopen(Request(link)).read()
        memory = BytesIO(remote)
        return memory


def scrape_page(url, page, dictionary):
    doc=get_pdf(url)
    prices_df=get_df(doc, page, dictionary)
    prices_df.reset_index(drop='index', inplace=True)
    return prices_df

def clean_prices(df):
    commissions={}
    for i in range(len(df)):
        if str(df['Prices'][i])=='nan':
            df['Prices'][i]='0'     
        else:# re.search(r'\d+,\d{2}/\d+,\d{2}', df['Prices'][i]):
            df['Prices'][i]='&'.join(re.findall(r'\d+,\d{2}', df['Prices'][i]))
        if df['Prices'][i]=='':
            df['Prices'][i]='0'
        if type(df['Commissions'][i])==str:
            if 'Nota' in df['Commissions'][i]:
                pass
            elif '\n-' in df['Commissions'][i]:
                names=df['Commissions'][i].split(sep='\n-')
                prices=['']+df['Prices'][i].split(sep='&')
                names=[n.replace('\n', '') for n in names]
                #prices=[p.replace('','0') for p in prices]
                commissions.update({names[0]:{}})
                commissions[names[0]].update({name: price for name, price in zip(names[1:], prices[1:])})
            elif '\n ' in df['Commissions'][i]:
                names=df['Commissions'][i].split(sep='\n ')
                prices=['']+df['Prices'][i].split(sep='&')
                names=[n.replace('\n', '') for n in names]
                #prices=[p.replace('','0') for p in prices]
                commissions.update({names[0]:{}})
                commissions[names[0]].update({name: price for name, price in zip(names[1:], prices[1:])})
            elif len(re.findall(r'\n[0-9]', df['Commissions'][i]))>0:
                names=re.split(r'\n[0-9]', df['Commissions'][i])
                prices=['']+df['Prices'][i].split(sep='&')
                names=[n.replace('\n', '') for n in names]
                #prices=[p.replace('','0') for p in prices]
                commissions.update({names[0]:{}})
                commissions[names[0]].update({name: price for name, price in zip(names[1:], prices[1:])})
            else:
                name = df['Commissions'][i]
                price = df['Prices'][i]
                if '&' in price:
                    price=df['Prices'][i].split(sep='&')[-1]
                name = name.replace('\n','').replace('\xa0','')
                commissions.update({name: price})
    return commissions

def scrape_all(url, pages, dictionary):
    house_cred={}
    if len(pages)>0:
        for page in pages:
            df=scrape_page(url, page, dictionary)
            house_cred.update({f'page {page}': clean_prices(df)})
        result=house_cred
    else:
        result='No housing credit product found in this bank'
    return result

In [9]:
url=bankinter[2]['link']
pages=bankinter[2]['page']
dictionary=house_credit_com
scrape_all(url, pages, dictionary)

{'page 8': {'Comissão de Estudo do Crédito Habitação': '260,00',
  'Comissão de Avaliação': '220,00',
  'Comissão de Formalização do Crédito Habitação': '120,00',
  'Comissão de Contrato do Crédito Habitação': '90,00',
  'Comissão de Tramitação': '90,00',
  'Comissão de Fiança': '120,00',
  'Comissão de Solicitadoria': '250,00',
  'Comissão de Vistoria': '150,00'},
 'page 9': {'Comissão de Processamento de Prestação': '2,50',
  'Comissão pela Recuperação de Valores em Dívida': '150,00',
  'Comissão de Penalização por Amortização Antecipada Parcial no Crédito Habitação:': {'   - Taxa Variável': '0'},
  'Comissão de Penalização por Amortização Antecipada Total no Crédito Habitação:': {'   - Taxa Variável': '0'}},
 'page 10': {'Comissão de Estudo e Análise (Arrangement Fee)': '240,00',
  'Comissão de Avaliação': '330,00',
  'Comissão de Tramitação': '90,00',
  'Comissão de Solicitadoria': '250,00',
  'Comissão de Vistoria': '225,00'},
 'page 11': {'Comissão de Processamento de Prestação':